In [27]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
import gym
import matplotlib.pyplot as plt
import time

In [28]:
# Hyperparameters
n_episodes = 100
batch_size = 10
gamma = 0.95
learning_rate = 0.1
epsilon_decay = 0.09
n_actions = 4  # Number of actions in FrozenLake

In [29]:
negative_reward_enabled = False
is_slippery_enabled = False

In [30]:
custom_map = [
    'SFFF',
    'FHFF',
    'FFHF',
    'HFGF'
]

In [31]:
# Initialize the environment
env = gym.make('FrozenLake-v1', desc=custom_map, is_slippery=is_slippery_enabled)
n_states = env.observation_space.n

# Create the Q-network model
model = keras.Sequential([
    keras.layers.InputLayer(batch_input_shape=(1, n_states)),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(n_actions, activation='linear')
])
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = keras.losses.mean_squared_error

In [32]:
model.compile(optimizer='adam', loss='mean_squared_error')

def one_hot(state, state_space=16):
    return np.identity(state_space)[state:state + 1].reshape(1, -1)

total_rewards = []
total_steps = []
success_rate = []

# Early stopping parameters
early_stopping_enabled = True
early_stopping_threshold = 0.75  # 75% average reward
early_stopping_check_every = 100  # check every 100 episodes
early_stopping_patience = 100  # patience counter (you may not need this depending on your early stopping criteria)

start_time = time.time()

episodes_to_train = n_episodes

for episode in tqdm(range(n_episodes)):
    state = env.reset()
    state = one_hot(state)  # One-hot encode the initial state
    done = False
    episode_reward = 0
    episode_steps = 0

    while not done:
        # Epsilon-greedy action selection
        epsilon = max(1 - episode * epsilon_decay / (n_episodes - 1), 0.01)
        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        else:
            q_values = model.predict(state)
            action = np.argmax(q_values[0])

        # Take action
        next_state, reward, done, _ = env.step(action)
        next_state = one_hot(next_state)  # One-hot encode the next state

        # Modify reward for the hole case if needed
        if done and reward == 0:
            reward = -1

        episode_reward += reward
        episode_steps += 1

        # Store in replay buffer and train
        target = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        target_f = model.predict(state)
        target_f[0][action] = target
        model.fit(state, target_f, epochs=1, verbose=0)

        # Update state
        state = next_state

    total_rewards.append(episode_reward)
    total_steps.append(episode_steps)
    success_rate.append(int(episode_reward > 0))

    # Print metrics every 100 episodes
    if (episode + 1) % 100 == 0:
        average_reward = np.mean(total_rewards[-100:])
        average_steps = np.mean(total_steps[-100:])
        success_percentage = np.mean(success_rate[-100:])
        print(f"Episode: {episode + 1}, Avg Reward: {average_reward}, Avg Steps: {average_steps}, Success Rate: {success_percentage * 100}%")

    # Early stopping check
    if early_stopping_enabled and (episode + 1) % early_stopping_check_every == 0:
        average_reward = np.mean(total_rewards[-early_stopping_check_every:])
        success_percentage = np.mean(success_rate[-early_stopping_check_every:])
        print(f"Episode: {episode + 1}, Avg Reward: {average_reward}, Success Rate: {success_percentage * 100}%")
        
        # Check if average reward is greater than the early stopping threshold
        if average_reward >= early_stopping_threshold:
            print(f"Early stopping triggered at episode {episode + 1}.")
            episodes_to_train = episode + 1
            break  # Break out of the loop to stop training

# End training timer
end_time = time.time()
training_time = end_time - start_time


  0%|          | 0/100 [00:00<?, ?it/s]

1/1 [==============================] - 0s 29ms/step


  1%|          | 1/100 [00:02<04:22,  2.65s/it]

1/1 [==============================] - 0s 21ms/step


  2%|▏         | 2/100 [00:06<05:35,  3.42s/it]

...

1/1 [==============================] - 0s 33ms/step


 99%|█████████▉| 99/100 [03:24<00:01,  1.78s/it]

1/1 [==============================] - 0s 30ms/step


100%|██████████| 100/100 [03:32<00:00,  2.12s/it]

Episode: 100, Avg Reward: -0.98, Avg Steps: 8.13, Success Rate: 1.0%
Episode: 100, Avg Reward: -0.98, Success Rate: 1.0%


In [15]:
# Post training metrics
overall_average_reward = np.mean(total_rewards)
overall_average_steps = np.mean(total_steps)
overall_success_rate = np.mean(success_rate)

print('----------------------------------------------------------')
print("Overall Average reward:", overall_average_reward)
print("Overall Average number of steps:", overall_average_steps)
print("Success rate (%):", overall_success_rate * 100)
print('----------------------------------------------------------')

----------------------------------------------------------
Overall Average reward: -0.96
Overall Average number of steps: 9.81
Success rate (%): 2.0
----------------------------------------------------------


In [19]:
def test_dqn_policy(env, model, num_tests=100):
    success_count = 0
    step_list = []  # To store the number of steps for successful episodes

    for _ in range(num_tests):
        state = env.reset()
        state = one_hot(state)  # Apply one-hot encoding to the initial state
        done = False
        step_count = 0
        
        while not done:
            q_values = model.predict(state)
            action = np.argmax(q_values[0])
            
            next_state, reward, done, _ = env.step(action)
            next_state = one_hot(next_state)  # Apply one-hot encoding to the next state
            step_count += 1
            
            # Assuming reward of 1 or higher indicates success
            if done and reward >= 1:
                success_count += 1
                step_list.append(step_count)
            
            state = next_state  # Update the state

    success_rate = success_count / num_tests
    average_steps_when_winning = np.mean(step_list) if step_list else float('inf')

    return success_rate, average_steps_when_winning

# After training your DQN model, you can call this function to evaluate it.
post_training_success_rate, average_steps_when_winning = test_dqn_policy(env, model)


1/1 [==============================] - 0s 21ms/step


KeyboardInterrupt: 

In [ ]:
print('')
print('Q-Learning with Negative reward: ', negative_reward_enabled, '; and slippery: ', is_slippery_enabled)
print('==========================================================')
print('The number of episodes', episodes_to_train)
print('Post-Training Success rate (%):', post_training_success_rate * 100)
print('Average number of steps when winning:', average_steps_when_winning)
print('Training Time (seconds):', training_time)
print('==========================================================')

In [ ]:
# Plotting metrics
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].plot(total_rewards, 'tab:green')
axs[0].set_title('Reward per Episode')
axs[1].plot(total_steps, 'tab:purple')
axs[1].set_title('Steps per Episode')
plt.show()